In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key= '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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [22]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-ii-itc/job/R-40715")
page_data = loader.load().pop().page_content
print(page_data)






















Software Engineer II, ITC










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Lan

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [25]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II, ITC',
 'experience': '2-5 years',
 'skills': ['Data engineering',
  'Python',
  'Java',
  'SQL',
  'Data Bricks',
  'Airflow',
  'EMR',
  'Hive',
  'Spark',
  'Data modelling',
  'ETL development',
  'Data streaming',
  'Microservices architecture',
  'Domain driven design',
  "RESTful API's",
  'Data management fundamentals',
  'Data storage principles',
  'Cloud services',
  'AWS',
  'CI/CD',
  'DevOps'],
 'description': 'We are looking for a Software engineer, to be a part of Nike’s Marketing Technology organization. This position will play a critical role, in developing Nike’s Customer Data platform, which supports and promotes our products and provides a unique user experience to all our customers in 166 countries globally.'}

In [26]:
type(json_res)

dict

In [27]:
job= json_res

In [15]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [28]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/full-stack-js-p

In [29]:
job['skills']

['Data engineering',
 'Python',
 'Java',
 'SQL',
 'Data Bricks',
 'Airflow',
 'EMR',
 'Hive',
 'Spark',
 'Data modelling',
 'ETL development',
 'Data streaming',
 'Microservices architecture',
 'Domain driven design',
 "RESTful API's",
 'Data management fundamentals',
 'Data storage principles',
 'Cloud services',
 'AWS',
 'CI/CD',
 'DevOps']

In [30]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Raghu, a business development executive at ABC. ABC is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABC 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ABC's portfolio: {link_list}
        Remember you are Raghu, BDE at ABC. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for Nike's Customer Data Platform

Dear Hiring Manager,

I came across the job posting for a Software Engineer II, ITC at Nike, and I'm excited to introduce ABC, an AI & Software Consulting company that can help you develop and implement a robust Customer Data platform. With our expertise in software engineering, we can support and promote Nike's products, providing a unique user experience to customers globally.

Our team at ABC has extensive experience in data engineering, Python, Java, SQL, Data Bricks, Airflow, EMR, Hive, Spark, data modeling, ETL development, data streaming, microservices architecture, domain-driven design, RESTful APIs, data management fundamentals, data storage principles, cloud services, AWS, CI/CD, and DevOps. We're confident that our skills align with the requirements of the job posting.

I'd like to highlight some of our relevant portfolio work:

* https://example.com/java-portfolio (Java portfolio)
* https://exa